In [1]:
# from datasets import load_dataset
# dataset = load_dataset("imagefolder", data_dir="/content/inaturalist_12K")
# dataset.push_to_hub("demoyolo/inaturalist-12k")


In [1]:
!pip install rustworkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
dataset = load_dataset("demoyolo/inaturalist-12k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/705 [00:00<?, ?B/s]

train-00000-of-00007.parquet:   0%|          | 0.00/475M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

...

train-00006-of-00007.parquet:   0%|          | 0.00/516M [00:00<?, ?B/s]

validation-00000-of-00002.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

validation-00001-of-00002.parquet:   0%|          | 0.00/325M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
import torch
device = "cpu"
dtype = torch.float32

In [5]:
# @title Dataset Class
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from datasets import load_dataset
from PIL import Image
import io

class iNaturalistDataset(Dataset):
    class_names = {
      '0': 'Amphibia',
      '1': 'Animalia',
      '2': 'Arachnida',
      '3': 'Aves',
      '4': 'Fungi',
      '5': 'Insecta',
      '6': 'Mammalia',
      '7': 'Mollusca',
      '8': 'Plantae',
      '9': 'Reptilia'
  }
    def __init__(self, hf_dataset, transform=None, split='train'):
        self.dataset = hf_dataset[split] if split in hf_dataset else hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        image = item['image']
        if not isinstance(image, Image.Image):
            if isinstance(image, bytes):
                image = Image.open(io.BytesIO(image))
            else:

                image = Image.fromarray(image)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = item['label'] #if 'label' in item else item.get('labels', 0)

        return image, label

# # Load the dataset
# dataset = load_dataset("demoyolo/inaturalist-12k")

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [6]:
# @title Autograd Graph
import rustworkx as rx
import weakref
class AutogradGraph:
    """
    Manages the computation graph for automatic differentiation.
    It uses a directed acyclic graph to track dependencies between tensors.
    """
    __slots__ = ('graph', 'intermediate_tensors', '_check_cycles', '_auto_cleanup', '__weakref__')

    def __init__(self, check_for_cycles=True, auto_cleanup=True):
        self.graph = rx.PyDiGraph()
        self.intermediate_tensors = {}
        self._check_cycles = check_for_cycles
        self._auto_cleanup = auto_cleanup

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self._check_cycles and self.check_cycle():
            raise RuntimeError("Cycle detected in autograd graph on context exit.")
        if self._auto_cleanup:
            self.intermediate_tensors.clear()
            self.graph.clear()

    def add_tensor_graph(self, tensor):
        if not tensor._custom_requires_grad:
            raise ValueError("Tensor with requires_grad=False cannot be added to the graph.")
        ref = weakref.proxy(tensor)
        tensor_index = self.graph.add_node(ref)
        tensor._node_id = tensor_index

    def add_non_leaf_tensor_reference(self, tensor):
        if not tensor._custom_requires_grad:
            raise ValueError("Tensor must require grad.")
        if tensor._node_id in self.intermediate_tensors:
            raise ValueError("Tensor reference already exists in intermediate tensors.")
        self.intermediate_tensors[tensor._node_id] = tensor

    def add_edge(self, node_from, node_to, weight=None):
        if not all(isinstance(n, int) for n in (node_from, node_to)):
            raise TypeError("Node indices must be integers.")
        if not self.graph.has_node(node_from) or not self.graph.has_node(node_to):
            raise ValueError("Nodes must exist before adding edge.")
        self.graph.add_edge(node_from, node_to, weight)

    def check_cycle(self):
        return not rx.is_directed_acyclic_graph(self.graph)

    def reverse_toposort_from_tensor(self, tensor_index):
        graph=self.graph
        predecessors = list(rx.ancestors(graph, tensor_index))
        predecessors.append(tensor_index)
        sub_graph = graph.subgraph(predecessors)
        return [sub_graph[i] for i in reversed(rx.topological_sort(sub_graph))]
    # def alternative_reverse_toposort_from_tensor(self, tensor_index):
    #     graph = self.graph
    #     relevant_nodes = rx.ancestors(graph, tensor_index)
    #     relevant_nodes.add(tensor_index)
    #     full_topo = rx.topological_sort(graph)
    #     relevant_topo = [graph[_node_id] for _node_id in reversed(full_topo) if _node_id in relevant_nodes]
    #     return relevant_topo

    def delete_node(self, node_index):
        if not isinstance(node_index, int):
            raise TypeError("Node index must be an integer.")
        if self.graph.has_node(node_index):
             self.graph.remove_node(node_index)
    def delete_edge(self, node_from, node_to):
        if not self.graph.has_edge(node_from, node_to):
            raise ValueError("Edge does not exist.")
        self.graph.remove_edge(node_from, node_to)

    def del_non_leaf_tensor_reference(self, tensor_node_id):
        self.intermediate_tensors.pop(tensor_node_id, None)

    def delete_all_non_leaf_nodes(self):
        # removes non leaf nodes from graph and clears the intermediate_tensors dict
        self.graph.remove_nodes_from(list(self.intermediate_tensors.keys()))
        for custom_tensor in self.intermediate_tensors.values():custom_tensor.clear()
        self.intermediate_tensors.clear()

    def __repr__(self):
        return f"CustomAutogradGraph(nodes={self.graph.num_nodes()}, edges={self.graph.num_edges()})"

if __name__ == "__main__":
    pass

In [7]:
# @title Custom Tensor
import torch
import torch.nn.functional as F
import weakref
import numbers
import math

class CustomTensor:
    """
    A custom tensor class that wraps a PyTorch tensor to enable a custom
    autograd engine. It tracks operations to build a computation graph.
    """
    __slots__ = ('tensor', '_node_id', '_custom_requires_grad', '_backward', 'graph', '__weakref__','_is_leaf')

    def __new__(cls, data, *, _custom_requires_grad=False, device=device, dtype=dtype, graph=None, due_to_operation=False, is_leaf=False):
        assert device is not None
        assert dtype is not None
        if isinstance(data, CustomTensor):
            return data  # Don't rewrap
        return super().__new__(cls)

    def __init__(self, data, *, _custom_requires_grad=False, device=device, dtype=dtype, graph=None, due_to_operation=False, is_leaf=False):
        if isinstance(data, CustomTensor):
            return

        self.tensor = data if due_to_operation else torch.as_tensor(data, dtype=dtype, device=device)
        self.tensor.requires_grad_(False)
        self._custom_requires_grad = _custom_requires_grad
        self._node_id = None
        self._backward = CustomTensor._empty_backward_hook
        self.graph = None
        self._is_leaf = is_leaf

        if _custom_requires_grad:
            self._init_graph(graph)

    def _init_graph(self, graph):
        if graph is None:
            raise ValueError("Graph must be provided if requires_grad is True.")
        is_leaf=self._is_leaf
        if is_leaf:
            self.graph = weakref.proxy(graph)
        else:
            self.graph = graph # this line is only reached for tensors which are created by operations and graph passed is already a weakreference hence no need for wrapping
        graph.add_tensor_graph(self)
        if not is_leaf:
            graph.add_non_leaf_tensor_reference(self)
    @staticmethod
    def _empty_backward_hook():
      """A picklable placeholder for the backward function."""
      return None

    def clear(self):
        # NEVER CALL FOR LEAF TENSORS
        if self._is_leaf: return # ideally this line should never execute, this is just a gaurd rail
        self.tensor.grad = None
        self._custom_requires_grad = False
        self._node_id = None
        self._backward = CustomTensor._empty_backward_hook#lambda: None
        self.graph = None

    def clear_full(self):
        self.tensor = None
        self._custom_requires_grad = False
        self._node_id = None
        self._backward = CustomTensor._empty_backward_hook
        self.graph = None

    def _zero_grad(self):
        """Sets the gradient of the underlying tensor to zero."""
        if self.tensor.grad is None:
            self.tensor.grad = torch.zeros_like(self.tensor)
        else:
            self.tensor.grad.zero_()

    def zero_(self):
        """Sets the gradient of the underlying tensor to zero."""
        if self.tensor.grad is not None:
            self.tensor.grad.zero_()

    def to(self, device, dtype=None):
        if dtype is None:
            dtype = self.tensor.dtype
        self.tensor = self.tensor.to(device, dtype)
        return self


    # --- Broadcasting Helper ---
    @torch.compile
    def _reduce_grad_for_broadcast(self, grad, target_shape):
        """Reduces a gradient to match the shape of a tensor that was broadcasted."""
        if grad.shape == target_shape:
            return grad

        # Add singleton dimensions to the front of target_shape to match grad's ndim
        padded_target_shape = (1,) * (grad.ndim - len(target_shape)) + target_shape

        # Identify dimensions that were broadcasted
        sum_dims = [i for i, (grad_dim, target_dim) in enumerate(zip(grad.shape, padded_target_shape)) if target_dim == 1 and grad_dim > 1]

        if sum_dims:
            grad = grad.sum(dim=sum_dims, keepdim=True)

        # Remove singleton dimensions to match the final target shape
        return grad.reshape(target_shape)



    def __add__(self, other):

        if isinstance(other, numbers.Number):
            return self._add_scalar(other)
        elif isinstance(other, CustomTensor):
            return self._add_tensor(other)
        return NotImplemented
    def __radd__(self,other):
        return self + other
    def __iadd__(self,other):
        if isinstance(other, numbers.Number):
            self.tensor.add_(other)
        elif isinstance(other,CustomTensor):
            self.tensor.add_(other.tensor)
    def _add_scalar(self, scalar):
        result_tensor = torch.add(self.tensor, scalar)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)
        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None: self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad)
        result._backward = _backward
        return result
    def _add_tensor(self, other):
        result_tensor = torch.add(self.tensor, other.tensor)
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)
        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None: self_ref._zero_grad()
                grad_for_self = self_ref._reduce_grad_for_broadcast(result_ref.tensor.grad, self_ref.tensor.shape)
                self_ref.tensor.grad.add_(grad_for_self)
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None: other_ref._zero_grad()
                grad_for_other = other_ref._reduce_grad_for_broadcast(result_ref.tensor.grad, other_ref.tensor.shape)
                other_ref.tensor.grad.add_(grad_for_other)
        result._backward = _backward
        return result

    def __mul__(self, other):
        if isinstance(other, numbers.Number):
            return self._mul_scalar(other)
        elif isinstance(other, CustomTensor):
            return self._mul_tensor(other)
        return NotImplemented
    def __rmul__(self,other):
        return self*other
    def __imul__(self,other):
        if isinstance(other, numbers.Number):
            self.tensor.mul_(other)
        elif isinstance(other,CustomTensor):
            self.tensor.mul_(other.tensor)
    def _mul_scalar(self, scalar):
        result_tensor = torch.mul(self.tensor, scalar)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)
        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad * scalar)
        result._backward = _backward
        return result
    def _mul_tensor(self, other):
        result_tensor = torch.mul(self.tensor, other.tensor)
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)
        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        result_ref = weakref.proxy(result)
        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)
        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None: self_ref._zero_grad()
                grad_for_self = self_ref._reduce_grad_for_broadcast(result_ref.tensor.grad * other_ref.tensor, self_ref.tensor.shape)
                self_ref.tensor.grad.add_(grad_for_self)
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None: other_ref._zero_grad()
                grad_for_other = other_ref._reduce_grad_for_broadcast(result_ref.tensor.grad * self_ref.tensor, other_ref.tensor.shape)
                other_ref.tensor.grad.add_(grad_for_other)
        result._backward = _backward
        return result

    def __sub__(self, other):
        if isinstance(other, numbers.Number):
            return self._sub_scalar(other)
        elif isinstance(other, CustomTensor):
            return self._sub_tensor(other)
        return NotImplemented

    def __rsub__(self, other):
        if isinstance(other, numbers.Number):
            return self._rsub_scalar(other)

    def __isub__(self,other):
        if isinstance(other, numbers.Number):
            self.tensor.sub_(other)
        elif isinstance(other,CustomTensor):
            self.tensor.sub_(other.tensor)

    def _rsub_scalar(self, scalar):
        result_tensor = torch.sub(scalar, self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            # Derivative of scalar - x is -1
            self_ref.tensor.grad.sub_(result_ref.tensor.grad) # No broadcasting specific logic for scalar op

        result._backward = _backward
        return result


    def _sub_scalar(self, scalar):
        result_tensor = torch.sub(self.tensor, scalar)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad) # No broadcasting specific logic for scalar op
        result._backward = _backward
        return result

    def _sub_tensor(self, other):
        result_tensor = torch.sub(self.tensor, other.tensor)
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)

        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        result_ref = weakref.proxy(result)

        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)

        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None:
                    self_ref._zero_grad()
                grad_for_self = self_ref._reduce_grad_for_broadcast(result_ref.tensor.grad, self_ref.tensor.shape)
                self_ref.tensor.grad.add_(grad_for_self)
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None:
                    other_ref._zero_grad()
                grad_for_other = other_ref._reduce_grad_for_broadcast(-result_ref.tensor.grad, other_ref.tensor.shape)
                other_ref.tensor.grad.add_(grad_for_other)
        result._backward = _backward
        return result

    def __truediv__(self, other):
        if isinstance(other, numbers.Number):
            return self._div_scalar(other)
        elif isinstance(other, CustomTensor):
            return self._div_tensor(other)
        return NotImplemented
    def __itruediv__(self,other):
        if isinstance(other, numbers.Number):
            self.tensor.div_(other)
        elif isinstance(other,CustomTensor):
            self.tensor.div_(other.tensor)
    def _div_scalar(self, scalar):
        result_tensor = torch.div(self.tensor, scalar)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad / scalar)
        result._backward = _backward
        return result

    def _div_tensor(self,other):
        result_tensor = torch.div(self.tensor, other.tensor)
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)

        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        result_ref = weakref.proxy(result)

        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)

        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None:
                    self_ref._zero_grad()
                grad_for_self = self_ref._reduce_grad_for_broadcast(result_ref.tensor.grad / other_ref.tensor, self_ref.tensor.shape)
                self_ref.tensor.grad.add_(grad_for_self)
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None:
                    other_ref._zero_grad()
                grad_for_other = other_ref._reduce_grad_for_broadcast(-result_ref.tensor.grad * self_ref.tensor / other_ref.tensor.pow(2), other_ref.tensor.shape)
                other_ref.tensor.grad.add_(grad_for_other)
        result._backward = _backward
        return result

    def pow(self, scalar):
        result_tensor = torch.pow(self.tensor, scalar)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            grad_contrib = scalar * self_ref.tensor.pow(scalar - 1)
            self_ref.tensor.grad.add_(result_ref.tensor.grad * grad_contrib)
        result._backward = _backward
        return result
    def __ipow__(self,other):
        self.tensor.pow_(other)
    def __pow__(self,other):
      if isinstance(other, numbers.Number):
          return self.pow(other)
      return NotImplemented
    def exp(self):
        out = torch.exp(self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(out,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(out, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad * out)
        result._backward = _backward
        return result

    def log(self):
        out = torch.log(self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(out,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(out, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad / self_ref.tensor)
        result._backward = _backward
        return result

    def sin(self):
        out = torch.sin(self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(out,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(out, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad * torch.cos(self_ref.tensor))
        result._backward = _backward
        return result

    def cos(self):
        out = torch.cos(self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(out,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(out, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(-result_ref.tensor.grad*torch.sin(self_ref.tensor))
        result._backward = _backward
        return result

    def sqrt(self):
        out = torch.sqrt(self.tensor)
        if not self._custom_requires_grad:
            return CustomTensor(out,due_to_operation=True)

        graph = self.graph
        result = CustomTensor(out, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)
        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)
        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad*0.5*self_ref.tensor.pow(-0.5))
        result._backward = _backward
        return result
    def __matmul__(self,other):
        if isinstance(other, CustomTensor):
            return self.matmul(other)
        return NotImplemented
    def matmul(self, other):
        result_tensor = torch.matmul(self.tensor, other.tensor)
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)

        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        result_ref = weakref.proxy(result)

        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)

        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None: self_ref._zero_grad()
                # Use robust broadcasting for matmul gradient
                grad_for_self = torch.matmul(result_ref.tensor.grad, other_ref.tensor.transpose(-2, -1))
                self_ref.tensor.grad.add_(self_ref._reduce_grad_for_broadcast(grad_for_self, self_ref.tensor.shape))
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None: other_ref._zero_grad()
                grad_for_other = torch.matmul(self_ref.tensor.transpose(-2, -1), result_ref.tensor.grad)
                other_ref.tensor.grad.add_(other_ref._reduce_grad_for_broadcast(grad_for_other, other_ref.tensor.shape))
        result._backward = _backward
        return result
    def dot(self, other):
        # torch.dot only works for 1D tensors, or for higher-D tensors,
        # it flattens them to 1D and then computes the dot product.
        # This means the gradients will also be 1D, so no complex broadcasting
        # reduction is needed on the output gradient itself.
        # However, the input tensors themselves could have been results of broadcasting ops.
        # For a truly general dot product, you'd use torch.matmul.
        result_tensor = torch.dot(self.tensor.reshape(-1), other.tensor.reshape(-1))
        requires_grad = self._custom_requires_grad or other._custom_requires_grad
        if not requires_grad:
            return CustomTensor(result_tensor,due_to_operation=True)

        graph = self.graph if self._custom_requires_grad else other.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)

        self_ref = weakref.proxy(self)
        other_ref = weakref.proxy(other)
        result_ref = weakref.proxy(result)

        if self._custom_requires_grad:
            graph.add_edge(self._node_id, result._node_id)
        if other._custom_requires_grad:
            graph.add_edge(other._node_id, result._node_id)

        def _backward():
            if self_ref._custom_requires_grad:
                if self_ref.tensor.grad is None:
                    self_ref._zero_grad()
                # The grad from result_ref.tensor.grad will be a scalar.
                # It needs to be multiplied by the other_ref.tensor (original shape)
                # and then potentially re-shaped if original was >1D
                grad_contrib = result_ref.tensor.grad * other_ref.tensor
                self_ref.tensor.grad.add_(grad_contrib)
            if other_ref._custom_requires_grad:
                if other_ref.tensor.grad is None:
                    other_ref._zero_grad()
                grad_contrib = result_ref.tensor.grad * self_ref.tensor
                other_ref.tensor.grad.add_(grad_contrib)
        result._backward = _backward
        return result



    # --- Unary Operations ---

    def sum(self, dim=None, keepdim=False):
        """Computes the sum of elements along given dimensions."""
        result_tensor = self.tensor.sum(dim=dim, keepdim=keepdim)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor, due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)

        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()

            grad = result_ref.tensor.grad
            # If keepdim was false, the summed dim was squeezed. We need to unsqueeze it back for broadcasting.
            if not keepdim and dim is not None:
                grad = grad.unsqueeze(dim)

            self_ref.tensor.grad.add_(grad)

        result._backward = _backward
        return result

    def mean(self, dim=None, keepdim=False):
        """Computes the mean of elements along given dimensions."""
        result_tensor = self.tensor.mean(dim=dim, keepdim=keepdim)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor, due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)

        # Determine the number of elements that were averaged
        if dim is None:
            n = self.tensor.numel()
        else:
            n = self.tensor.shape[dim]

        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()

            grad = result_ref.tensor.grad
            if not keepdim and dim is not None:
                grad = grad.unsqueeze(dim)

            # Distribute gradient evenly
            self_ref.tensor.grad.add_(grad / n)

        result._backward = _backward
        return result

    def reshape(self, *shape):
        """Reshapes the tensor to the given shape."""
        original_shape = self.shape
        result_tensor = self.tensor.reshape(*shape)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor, due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)

        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            self_ref.tensor.grad.add_(result_ref.tensor.grad.reshape(original_shape))

        result._backward = _backward
        return result

    def transpose(self, dim0, dim1):
        """Transposes dimensions dim0 and dim1."""
        result_tensor = self.tensor.transpose(dim0, dim1)
        if not self._custom_requires_grad:
            return CustomTensor(result_tensor, due_to_operation=True)

        graph = self.graph
        result = CustomTensor(result_tensor, _custom_requires_grad=True, graph=graph, due_to_operation=True, is_leaf=False)
        graph.add_edge(self._node_id, result._node_id)

        self_ref = weakref.proxy(self)
        result_ref = weakref.proxy(result)

        def _backward():
            if self_ref.tensor.grad is None:
                self_ref._zero_grad()
            # The gradient operation for transpose is another transpose
            self_ref.tensor.grad.add_(result_ref.tensor.grad.transpose(dim0, dim1))

        result._backward = _backward
        return result

    @property
    def T(self):
        """Alias for transpose(-2, -1) for 2D or higher dimensional tensors."""
        if self.ndim < 2:
            raise ValueError("`.T` is only supported on tensors with 2 or more dimensions.")
        return self.transpose(-2, -1)

    def backward(self, weightage_tensor=1,retain_graph=False):
        if not self._custom_requires_grad:
            raise RuntimeError("Output tensor does not require grad.")
        if self.graph is None:
            raise RuntimeError("Output tensor is not part of a graph.")
        graph = self.graph

        # Initialize gradient for the output tensor
        if isinstance(weightage_tensor, numbers.Number):
            self.tensor.grad = torch.full_like(self.tensor, fill_value=weightage_tensor)
        elif isinstance(weightage_tensor, torch.Tensor):
            self.tensor.grad = weightage_tensor.to(self.tensor.device)#.clone()

        nodes_to_process = graph.reverse_toposort_from_tensor(self._node_id)

        for tensor_node in nodes_to_process:
            tensor_node._backward()
        if not retain_graph:
            graph.delete_all_non_leaf_nodes()

            #try:
                # The node is a weakref.proxy, check if it's still alive
                #if tensor_node.__class__ is weakref.ProxyType:
            #        tensor_node._backward()
            # except ReferenceError:
            #     # The tensor object was garbage collected, skip.
            #     print("dead reference node encountered")
            #     continue
    # --- Properties and Dunder Methods ---
    @property
    def dtype(self): return self.tensor.dtype
    @property
    def ndim(self): return self.tensor.ndim
    @property
    def shape(self): return self.tensor.shape
    @property
    def grad(self): return self.tensor.grad
    def __repr__(self): return f"CustomTensor({self.tensor}, grad_fn={self._backward != None}, requires_grad={self._custom_requires_grad})"
    # def __del__(self):
    #     if self._node_id is not None and self._is_leaf:
    #         try:
    #             if self.graph: self.graph.delete_node(self._node_id)
    #         except ReferenceError: # Graph might be gone first
    #             pass
if __name__ == "__main__":
    pass

In [8]:
# @title Modules
import torch
import math
import weakref
import torch.nn.functional as F
from collections import OrderedDict
class Module:
    """
    Base class for all neural network modules.
    """
    device=device
    dtype=dtype
    __slots__ = ('_parameters', '_modules', 'training')
    def __init__(self):
        self._parameters = OrderedDict()
        self._modules = OrderedDict()
        # self._buffers = OrderedDict()
        self.training = True #

    def __setattr__(self, name, value):
        if isinstance(value, CustomTensor):
            if value._custom_requires_grad:
                self._parameters[name] = value
        elif isinstance(value, Module):
            self._modules[name] = value
        # Handle buffers (non-parameter tensors like running_mean in BatchNorm)
        # elif isinstance(value, torch.Tensor):
        #     self._buffers[name] = value
        super().__setattr__(name, value)

    def parameters(self):
        """Returns a list of all parameters in the module and its submodules."""
        params = list(self._parameters.values())
        for module in self._modules.values():
            params.extend(module.parameters())
        return params

    def zero_grad(self):
        """Sets gradients of all model parameters to zero."""
        for p in self.parameters():
            p._zero_grad()

    def train(self):
        """Sets the module and all its submodules to training mode."""
        self.training = True
        for module in self._modules.values():
            module.train()

    def eval(self):
        """Sets the module and all its submodules to evaluation mode."""
        self.training = False
        for module in self._modules.values():
            module.eval()

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def to(self,device,dtype=None):
        if dtype is None:
            dtype = Module.dtype
        for p in self.parameters():
            p.to(device=device,dtype=dtype)
        return self


    def forward(self, *args, **kwargs):
        raise NotImplementedError("Subclasses of Module must implement a forward method.")

class Linear(Module):
    """Applies a linear transformation to the incoming data: y = xA^T + b
    types of activation relu,leaky_relu, gelu, sigmoid, tanh, silu,elu"""
    __slots__ = ('in_features', 'out_features', 'graph', 'weight', 'bias','__weakref__')
    _ACTIVATION_INIT = {
        "relu": ("kaiming_uniform_", "relu"),
        "gelu": ("kaiming_uniform_", "relu"),
        "silu": ("kaiming_uniform_", "relu"),
        "elu": ("kaiming_uniform_", "relu"),
        "gelu_approx": ("kaiming_uniform_", "relu"),
        "leaky_relu": ("kaiming_uniform_", "leaky_relu"),
        "sigmoid": ("xavier_uniform_", 1.0),
        "tanh": ("xavier_uniform_", 5/3)
    }

    def __new__(cls, in_features, out_features, bias=True, *, graph=None, activation="relu"):
        assert activation in cls._ACTIVATION_INIT
        return super().__new__(cls)

    def __init__(self, in_features, out_features, bias=True, *, graph=None, activation="relu"):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.graph = weakref.proxy(graph) if graph is not None else None

        # Initialize weight
        self.weight = CustomTensor(torch.empty(out_features, in_features, device=Linear.device, dtype=Linear.dtype),
                                 _custom_requires_grad=True, graph=graph, is_leaf=True)

        init_method, init_param = self._ACTIVATION_INIT[activation]
        if init_method == "kaiming_uniform_":
            torch.nn.init.kaiming_uniform_(self.weight.tensor, nonlinearity=init_param)
        else:  # xavier_uniform_
            torch.nn.init.xavier_uniform_(self.weight.tensor, gain=init_param)

        # Initialize bias
        self.bias = CustomTensor(torch.zeros(out_features,device=Linear.device, dtype=Linear.dtype),
                               _custom_requires_grad=True, graph=graph, is_leaf=True) if bias else None

    def forward(self, input_tensor):
        inp = input_tensor.tensor
        is_1d = inp.ndim==1
        if is_1d:
            inp = inp.unsqueeze(0)
        output = inp @ self.weight.tensor.transpose(-2, -1)
        if self.bias is not None:
            output.add_(self.bias.tensor)

        if is_1d:
            output = output.squeeze(0)
        if not self.training:
            return CustomTensor(output, due_to_operation=True)

        # Training mode - setup gradient computation
        result = CustomTensor(output, _custom_requires_grad=True, graph=self.graph,
                            due_to_operation=True, is_leaf=False)

        # Add edges to computation graph
        if input_tensor._custom_requires_grad:
            self.graph.add_edge(input_tensor._node_id, result._node_id)
        self.graph.add_edge(self.weight._node_id, result._node_id)
        if self.bias is not None:
            self.graph.add_edge(self.bias._node_id, result._node_id)

        # Create weak references for backward pass
        refs = {
            'weight': weakref.proxy(self.weight),
            'input': weakref.proxy(input_tensor),
            'result': weakref.proxy(result),
            'bias': weakref.proxy(self.bias) if self.bias is not None else None,
            'is_1d': is_1d
        }

        result._backward = self._create_backward(refs)
        return result

    def _create_backward(self, refs):
        def _backward():
            weight_ref, input_ref, result_ref, bias_ref, is_1d = refs['weight'], refs['input'], refs['result'], refs['bias'], refs['is_1d']
            grad_output = result_ref.tensor.grad
            inp = input_ref.tensor
            if is_1d:
                inp = inp.unsqueeze(0)
                grad_output = grad_output.unsqueeze(0)

            # Weight gradient
            if weight_ref._custom_requires_grad:
                if weight_ref.tensor.grad is None:
                    weight_ref._zero_grad()
                grad_w = torch.matmul(grad_output.transpose(-2, -1), inp)
                weight_ref.tensor.grad.add_(weight_ref._reduce_grad_for_broadcast(grad_w, weight_ref.tensor.shape))

            # Bias gradient
            if bias_ref is not None and bias_ref._custom_requires_grad:
                if bias_ref.tensor.grad is None:
                    bias_ref._zero_grad()
                grad_b = bias_ref._reduce_grad_for_broadcast(grad_output, bias_ref.tensor.shape)
                bias_ref.tensor.grad.add_(grad_b)

            # Input gradient
            if input_ref._custom_requires_grad:
                if input_ref.tensor.grad is None:
                    input_ref._zero_grad()
                grad_in = torch.matmul(grad_output, weight_ref.tensor)
                if is_1d:
                    grad_in = grad_in.squeeze(0)
                input_ref.tensor.grad.add_(input_ref._reduce_grad_for_broadcast(grad_in, input_ref.tensor.shape))

        return _backward

class Conv2d(Module):
    """Applies a 2D convolution over an input signal composed of several input planes.
    types of activation relu,leaky_relu, gelu, sigmoid, tanh, silu,elu"""
    __slots__ = ('in_channels', 'out_channels', 'kernel_size', 'stride', 'dilation', 'padding', 'groups', 'graph', 'weight', 'bias','__weakref__')

    # Lookup table for activation initialization
    _ACTIVATION_INIT = {
        "relu": ("kaiming_uniform_", "relu"),
        "gelu": ("kaiming_uniform_", "relu"),
        "silu": ("kaiming_uniform_", "relu"),
        "elu": ("kaiming_uniform_", "relu"),
        "gelu_approx": ("kaiming_uniform_", "relu"),
        "leaky_relu": ("kaiming_uniform_", "leaky_relu"),
        "sigmoid": ("xavier_uniform_", 1.0),
        "tanh": ("xavier_uniform_", 5/3)
    }

    def __new__(cls, *,in_channels, out_channels, kernel_size, stride=1,dilation=1,groups=1,bias=True, padding=0, graph=None,activation="relu"):
        assert isinstance(kernel_size, int) or len(kernel_size) == 2
        assert isinstance(stride, int) or len(stride) == 2
        assert isinstance(dilation, int) or len(dilation) == 2
        assert isinstance(padding, int) or len(padding) == 2
        assert activation in cls._ACTIVATION_INIT
        return super().__new__(cls)

    def __init__(self, *,in_channels, out_channels, kernel_size, stride=1,dilation=1,groups=1,bias=True, padding=0, graph=None,activation="relu"):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.dilation = (dilation, dilation) if isinstance(dilation, int) else dilation
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.groups = groups
        self.graph = weakref.proxy(graph) if graph is not None else None

        weight_shape = (out_channels, in_channels // groups, *self.kernel_size)
        self.weight = CustomTensor(torch.empty(weight_shape,device=Conv2d.device,dtype=Conv2d.dtype), _custom_requires_grad=True, graph=graph, is_leaf=True)

        # Use lookup table for initialization
        init_method, init_param = self._ACTIVATION_INIT[activation]
        if init_method == "kaiming_uniform_":
            torch.nn.init.kaiming_uniform_(self.weight.tensor, nonlinearity=init_param)
        else:  # xavier_uniform_
            torch.nn.init.xavier_uniform_(self.weight.tensor, gain=init_param)

        self.bias = CustomTensor(torch.zeros(out_channels,device=Conv2d.device,dtype=Conv2d.dtype), _custom_requires_grad=True, graph=graph, is_leaf=True) if bias else None

    def forward(self, input_tensor):
        output_tensor = F.conv2d(
            input = input_tensor.tensor,
            weight = self.weight.tensor,
            bias = self.bias.tensor if self.bias else None,
            stride = self.stride,
            padding = self.padding,
            groups=self.groups
        )
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph, due_to_operation=True, is_leaf=False)

        self.graph.add_edge(input_tensor._node_id, result._node_id)
        self.graph.add_edge(self.weight._node_id, result._node_id)
        if self.bias is not None:
            self.graph.add_edge(self.bias._node_id, result._node_id)

        # Create weak references for backward pass
        refs = {
            'input': weakref.proxy(input_tensor),
            'weight': weakref.proxy(self.weight),
            'bias': weakref.proxy(self.bias) if self.bias is not None else None,
            'result': weakref.proxy(result)
        }

        result._backward = self._create_backward(refs)
        return result

    def _create_backward(self, refs):
        def _backward():
            input_ref, weight_ref, bias_ref, result_ref = refs['input'], refs['weight'], refs['bias'], refs['result']
            grad_output = result_ref.tensor.grad

            if bias_ref is not None:
                if bias_ref._custom_requires_grad:
                    if bias_ref.tensor.grad is None: bias_ref._zero_grad()
                    bias_ref.tensor.grad.add_(grad_output.sum(dim=[0, 2, 3]))

            if input_ref._custom_requires_grad:
                if input_ref.tensor.grad is None: input_ref._zero_grad()
                input_ref.tensor.grad.add_(
                    self._calculate_gradient_input_tensor(input_ref.tensor,weight_ref.tensor,grad_output)
                )

            if weight_ref._custom_requires_grad:
                if weight_ref.tensor.grad is None: weight_ref._zero_grad()
                # tried vectorizing groups but failed hence using autograd for computing weight for efficiency (NOTE This is considered cheating)
                weight_ref.tensor.grad.add_(
                    torch.nn.grad.conv2d_weight(
                    input=input_ref.tensor,
                    weight_size=weight_ref.tensor.shape,
                    grad_output=grad_output,
                    stride=self.stride,
                    padding=self.padding,
                    dilation=self.dilation,
                    groups=self.groups
                    )
                )
        return _backward

    @torch.compile
    def _calculate_gradient_input_tensor(self, input_tensor,weight_tensor,grad_output):
        h_in, w_in = input_tensor.shape[2], input_tensor.shape[3]
        h_out, w_out = grad_output.shape[2], grad_output.shape[3]
        stride = self.stride
        padding = self.padding
        kernel_size = self.kernel_size
        dilation = self.dilation
        # The formula relating input size to output size in a transposed convolution is:
        # InputSize = (OutputSize - 1) * stride - 2 * padding + dilation * (kernel - 1) + output_padding + 1
        # We rearrange this to solve for the required output_padding.
        output_padding_h = h_in - ((h_out - 1) * stride[0] - 2 * padding[0] + dilation[0] * (kernel_size[0] - 1) + 1)
        output_padding_w = w_in - ((w_out - 1) * stride[1] - 2 * padding[1] + dilation[1] * (kernel_size[1] - 1) + 1)
        output_padding = (output_padding_h, output_padding_w)

        grad_input = F.conv_transpose2d(
            grad_output,
            weight_tensor,
            stride=stride,
            padding=padding,
            output_padding=output_padding,
            dilation=dilation,
            groups=self.groups
        )
        return grad_input

    @torch.compile
    def _calculate_gradient_weight_tensor_loop(self,input_tensor,grad_output):
        #The gradient w.r.t. the weights is a convolution
        # of the input (X) and the output gradient (grad_output).
        # For grouped convolutions, we must perform this calculation for each group separately.
        #O(b,co,oh,ow)=B(co)+ kh =0∑KH −1  kw =0∑KW −1  ci=(co/G)⋅(Cin/G)∑((co/G)+1)⋅(Cin/G)−1
        #  Ipadded(b,ci,ih,iw)K(co ,ci ,kh ,kw ),
        # where ih  = oh.sh+kh.dh, iw = ow.sw+kw.dw

        # ∂L/∂K(ci′ ,co′ ,kh′ ,kw′ ) =b,oh,ow∑ G(b,co',oh,ow)
        # Ipadded(b,ci', oh.sh + kh'.dh, ow.sw + kw'.dw)

        # the original operation is a summation over kh and kw and the input image
        # coordinates ih iw are sampled with dilation. (oh and ow for individual coordinates are constant)


        # the equation for the gradient is a summation over oh and ow and the input image
        # coordinates ih iw are sampled with stride.
        # (kh and kw are constant for individual coordinates are constant)

        # hence when calling conv2d we need to switch stride and dilation
        # and also transpose the dimensions of batch and channel as for derivative with respect to weight the channels are fixed in the summation

        in_channels = self.in_channels
        groups = self.groups
        out_channels = self.out_channels
        in_channels_per_group = in_channels // groups
        out_channels_per_group = out_channels // groups
        grad_W_groups = []

        for g in range(groups):
            # Slice the input tensor to get the channels for the current group
            start_in_ch = g * in_channels_per_group
            end_in_ch = start_in_ch + in_channels_per_group
            X_g = input_tensor[:, start_in_ch:end_in_ch, :, :]

            # Slice the output gradient tensor to get the channels for the current group
            start_out_ch = g * out_channels_per_group
            end_out_ch = start_out_ch + out_channels_per_group
            grad_output_g = grad_output[:, start_out_ch:end_out_ch, :, :]

            # To calculate the weight gradient via a convolution, we must cleverly
            # permute the input (X_g) and output gradient (grad_output_g) tensors.
            # We treat X_g as the input and grad_output_g as the kernel.
            # X_g: (N, Cin/g, H, W) -> permute -> (Cin/g, N, H, W)
            # grad_output_g: (N, Cout/g, oH, oW) -> permute -> (Cout/g, N, oH, oW)
            # The F.conv2d call then treats 'Cin/g' as the batch size and 'N' as the input channels.
            # The stride and dilation parameters from the original convolution are swapped.
            X_g_permuted = X_g.transpose(0, 1)
            grad_output_g_permuted = grad_output_g.transpose(0, 1)

            grad_W_g_permuted = F.conv2d(
                X_g_permuted,
                grad_output_g_permuted,
                stride=self.dilation,
                padding=self.padding,
                dilation=self.stride,
                groups=1 # The group calculation is handled by our loop, so this is a standard conv.
            )

            # The result has shape (Cin/g, Cout/g, kH, kW). We must permute it back to
            # the standard weight layout of (Cout/g, Cin/g, kH, kW).
            grad_W_g = grad_W_g_permuted.transpose(0, 1)
            grad_W_groups.append(grad_W_g)

        # Concatenate the gradients from all groups along the output channel dimension.
        # The weight tensor for grouped convolutions is laid out by stacking the weights
        # for each group, so we do the same for the gradient.
        grad_weight = torch.cat(grad_W_groups, dim=0)
        return grad_weight

    # def _calculate_gradient_weight_tensor_cheating(self,input_tensor,grad_output):
    #     return torch.nn.grad.conv2d_weight(
    #     input=input_tensor,
    #     weight_size=self.weight.tensor.shape,
    #     grad_output=grad_output,
    #     stride=self.stride,
    #     padding=self.padding,
    #     dilation=self.dilation,
    #     groups=self.groups
    #     )

class BatchNorm_Nd(Module):
    __slots__ = ('num_features', 'eps', 'momentum', 'graph', 'weight', 'bias', 'running_mean', 'running_var', '_channel_axis', '_shape_cache','__weakref__')
    def __new__(cls, num_features, eps=1e-5, momentum=0.1, *, graph=None):
        assert num_features > 0
        return super().__new__(cls)

    def __init__(self, num_features, eps=1e-5, momentum=0.1, *, graph=None):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.graph = weakref.proxy(graph)

        self.weight = CustomTensor(torch.ones(num_features,device=BatchNorm_Nd.device,dtype=BatchNorm_Nd.dtype), _custom_requires_grad=True, graph=graph, is_leaf=True)
        self.bias = CustomTensor(torch.zeros(num_features,device=BatchNorm_Nd.device,dtype=BatchNorm_Nd.dtype), _custom_requires_grad=True, graph=graph, is_leaf=True)

        self.running_mean = torch.zeros(num_features,device=BatchNorm_Nd.device,dtype=BatchNorm_Nd.dtype)
        self.running_var = torch.ones(num_features,device=BatchNorm_Nd.device,dtype=BatchNorm_Nd.dtype)

        self._channel_axis = 1
        self._shape_cache = {}

    def _get_broadcast_shape(self, input_shape):
        if input_shape not in self._shape_cache:
            self._shape_cache[input_shape] = (1,) + (input_shape[1],) + (1,) * (len(input_shape) - 2)
        return self._shape_cache[input_shape]

    @torch.compile
    def _compute_stats(self, x: torch.Tensor):
        reduce_dims = tuple(i for i in range(x.dim()) if i != self._channel_axis)

        mean = x.mean(dim=reduce_dims, keepdim=False)
        var = x.var(dim=reduce_dims, keepdim=False, unbiased=False)

        return mean, var

    def _create_backward(self, input_tensor, result, torch_input_tensor, normalized,
                        shape_to, weight_shaped, input_minus_mean, inv_std, total_elements):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)
        weight_ref = weakref.proxy(self.weight)
        bias_ref = weakref.proxy(self.bias)

        def _backward():
            result_gradient = result_ref.tensor.grad
            reduce_dims = tuple(i for i in range(input_ref.tensor.dim()) if i != self._channel_axis)
            if bias_ref._custom_requires_grad:
                if bias_ref.tensor.grad is None:
                    bias_ref._zero_grad()
                grad_bias = result_gradient.sum(dim=reduce_dims)
                bias_ref.tensor.grad.add_(grad_bias.view(bias_ref.tensor.shape))

            if weight_ref._custom_requires_grad:
                if weight_ref.tensor.grad is None:
                    weight_ref._zero_grad()
                grad_weight = (result_gradient * normalized).sum(dim=reduce_dims)
                weight_ref.tensor.grad.add_(grad_weight.view(weight_ref.tensor.shape))

            if input_ref._custom_requires_grad:
                if input_ref.tensor.grad is None:
                    input_ref._zero_grad()
                grad_input = self.batchnorm_gradient_for_input_tensor(
                    result_gradient=result_gradient,
                    input_tensor=torch_input_tensor,
                    weight_shaped=weight_shaped,
                    input_minus_mean=input_minus_mean,
                    inv_std=inv_std,
                    total_elements=total_elements
                )
                input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        torch_input_tensor = input_tensor.tensor
        shape_to = self._get_broadcast_shape(torch_input_tensor.shape)

        # Pre-compute shaped tensors once
        weight_shaped = self.weight.tensor.view(shape_to)
        bias_shaped = self.bias.tensor.view(shape_to)

        if self.training:
            batch_mean, batch_var = self._compute_stats(torch_input_tensor)
            total_elements = torch_input_tensor.numel() // torch_input_tensor.shape[self._channel_axis]
            unbiased_var = batch_var * total_elements / (total_elements - 1) if total_elements > 1 else batch_var

            # Update running statistics in-place
            self.running_mean.mul_(1-self.momentum).add_(batch_mean, alpha=self.momentum)
            self.running_var.mul_(1-self.momentum).add_(unbiased_var, alpha=self.momentum)

            mean, var = batch_mean, batch_var
        else:
            mean, var = self.running_mean, self.running_var
            mean_shaped = mean.view(shape_to)
            var_shaped = var.view(shape_to)
            normalized = (torch_input_tensor - mean_shaped) / torch.sqrt(var_shaped + self.eps)
            result = normalized * weight_shaped + bias_shaped
            return CustomTensor(result, due_to_operation=True)

        # Forward pass computation (training mode)
        mean_shaped = mean.view(shape_to)
        var_shaped = var.view(shape_to)

        inv_std = torch.rsqrt(var_shaped + self.eps)
        input_minus_mean = torch_input_tensor - mean_shaped
        normalized = input_minus_mean * inv_std
        output = normalized * weight_shaped + bias_shaped

        result = CustomTensor(output, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)

        # Build computation graph
        graph = self.graph
        graph.add_edge(input_tensor._node_id, result._node_id)
        graph.add_edge(self.weight._node_id, result._node_id)
        graph.add_edge(self.bias._node_id, result._node_id)

        # Create and assign backward function
        result._backward = self._create_backward(
            input_tensor, result, torch_input_tensor, normalized,
            shape_to, weight_shaped, input_minus_mean, inv_std, total_elements
        )

        return result

    @torch.compile
    def batchnorm_gradient_for_input_tensor(self, *, result_gradient, input_tensor, weight_shaped,
                                          input_minus_mean, inv_std, total_elements):
        reduce_dims = tuple(i for i in range(input_tensor.dim()) if i != self._channel_axis)

        outer_term = weight_shaped * inv_std
        term_1 = result_gradient
        term_2 = (-1/total_elements) * result_gradient.sum(dim=reduce_dims, keepdim=True)
        term3_sum_component = (input_minus_mean * result_gradient).sum(dim=reduce_dims, keepdim=True)
        term3 = inv_std**2 * (-1/total_elements) * input_minus_mean * term3_sum_component
        return outer_term * (term_1 + term_2 + term3)

class MaxPool2d(Module):
    __slots__ = ('kernel_size', 'stride', 'dilation', 'padding', 'graph','__weakref__')
    def __new__(cls, *, kernel_size, stride=1, padding=0, dilation=1, graph=None):
        assert isinstance(kernel_size, int) or len(kernel_size) == 2
        assert isinstance(stride, int) or len(stride) == 2
        assert isinstance(dilation, int) or len(dilation) == 2
        assert isinstance(padding, int) or len(padding) == 2
        return super().__new__(cls)

    def __init__(self, *, kernel_size, stride=1, padding=0, dilation=1, graph=None):
        super().__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.dilation = (dilation, dilation) if isinstance(dilation, int) else dilation
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.graph = weakref.proxy(graph) if graph is not None else None

    def _create_backward(self, input_tensor, result, cached_indices):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            input = input_ref.tensor
            grad_input = MaxPool2d._calculate_gradient_input_tensor(grad_output, cached_indices, input)
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        kernel_size = self.kernel_size
        stride = self.stride
        padding = self.padding
        dilation = self.dilation

        output_tensor, max_indices = F.max_pool2d(
            input=input_tensor.tensor,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            return_indices=True
        )

        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        graph = self.graph
        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=graph,due_to_operation=True, is_leaf=False)
        graph.add_edge(input_tensor._node_id, result._node_id)


        result._backward = self._create_backward(input_tensor, result, max_indices)

        return result
    @staticmethod
    @torch.compile
    def _calculate_gradient_input_tensor(grad_output, indices, input):
      # grad_output: (N, C, H_out, W_out)
      # indices:     (N, C, H_out, W_out)
      N, C, H_out, W_out = grad_output.shape
      # Initialize grad_input
      grad_input = torch.zeros_like(input)
      # Flatten spatial dims
      grad_output_flat = grad_output.view(N, C, -1)
      indices_flat = indices.view(N, C, -1)
      grad_input_flat = grad_input.view(N, C, -1)
      # Scatter gradients into appropriate positions
      grad_input_flat.scatter_add_(2, indices_flat, grad_output_flat)
      # Reshape back to input shape
      grad_input = grad_input_flat.view(input.shape)
      return grad_input

    def __repr__(self):
        return f"MaxPool2d(kernel_size={self.kernel_size}, stride={self.stride}, padding={self.padding})"

class AvgPool2d(Module):
    __slots__ = ('kernel_size', 'stride', 'padding', 'graph','__weakref__')
    def __new__(cls, *, kernel_size, stride=1, padding=0, graph=None):
        assert isinstance(kernel_size, int) or len(kernel_size) == 2
        assert isinstance(stride, int) or len(stride) == 2
        assert isinstance(padding, int) or len(padding) == 2
        return super().__new__(cls)

    def __init__(self, *, kernel_size, stride=1, padding=0, graph=None):
        super().__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.graph = weakref.proxy(graph) if graph is not None else None

    def create_backward(self, input_tensor, result):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            input = input_ref.tensor
            grad_input = self._calculate_gradient_input_tensor(grad_output,input)
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        kernel_size = self.kernel_size
        stride = self.stride
        padding = self.padding

        output_tensor = F.avg_pool2d(
            input=input_tensor.tensor,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            count_include_pad=True
        )

        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph, due_to_operation=True, is_leaf=False)
        graph = self.graph
        graph.add_edge(input_tensor._node_id, result._node_id)

        result._backward = self.create_backward(input_tensor, result)

        return result

    @torch.compile
    def _calculate_gradient_input_tensor(self,grad_output,input):

            h_in, w_in = input.shape[2], input.shape[3]
            h_out, w_out = grad_output.shape[2], grad_output.shape[3]
            kernel_size=self.kernel_size
            stride=self.stride
            padding=self.padding

            # The formula relating input size to output size in a transposed convolution is:
            # InputSize = (OutputSize - 1) * stride - 2 * padding + dilation * (kernel - 1) + output_padding + 1
            # We rearrange this to solve for the required output_padding.
            output_padding_h = h_in - ((h_out - 1) * stride[0] - 2 * padding[0] +  (kernel_size[0] - 1) + 1)
            output_padding_w = w_in - ((w_out - 1) * stride[1] - 2 * padding[1] +  (kernel_size[1] - 1) + 1)
            output_padding = (output_padding_h, output_padding_w)
            pool_size = kernel_size[0] * kernel_size[1]
            grad_kernel = torch.ones(grad_output.shape[1], 1, kernel_size[0], kernel_size[1],device=grad_output.device,dtype=grad_output.dtype) / pool_size
            grad_input = F.conv_transpose2d(
                input= grad_output,
                weight = grad_kernel,
                stride = stride,
                padding = padding,
                output_padding=output_padding,
                groups = input.shape[1]
            )
            return grad_input

class ReLu(Module):
    __slots__ = ('graph','__weakref__')
    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def _create_backward(self, input_tensor, result):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = grad_output.clone()
            grad_input[input_ref.tensor <= 0] = 0
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.relu(input_tensor.tensor)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result)
        return result

class Leaky_ReLu(Module):
    __slots__ = ('graph', 'negative_slope', '__weakref__')
    def __new__(cls, *, negative_slope=0.01, graph=None):
        assert negative_slope > 0
        return super().__new__(cls)

    def __init__(self, *, negative_slope=0.01, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None
        self.negative_slope = negative_slope

    def _create_backward(self, input_tensor, result):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = grad_output.clone()
            grad_input[input_ref.tensor <= 0] *= self.negative_slope
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.leaky_relu(input_tensor.tensor, negative_slope=self.negative_slope)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result)
        return result

class Elu(Module):
    __slots__ = ('graph', 'alpha', '__weakref__')
    def __new__(cls, *, alpha=1.0, graph=None):
        assert alpha > 0
        return super().__new__(cls)

    def __init__(self, *, alpha=1.0, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None
        self.alpha = alpha

    def _create_backward(self, input_tensor, result, output_tensor):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = grad_output.clone()
            mask_neg = (input_ref.tensor.data <= 0)
            grad_input[mask_neg] *= (self.alpha + output_tensor[mask_neg])
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.elu(input_tensor.tensor, alpha=self.alpha)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result, output_tensor)
        return result

class GeLu(Module):
    __slots__ = ('graph', 'approximate', '__weakref__')
    def __new__(cls, *, approximate='none', graph=None):
        assert approximate in {"none", "tanh"}
        return super().__new__(cls)

    def __init__(self, *, approximate='none', graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None
        self.approximate = approximate

    def _create_backward(self, input_tensor, result):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = GeLu.gelu_derivative(input_ref.tensor, grad_output, self.approximate)
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.gelu(input_tensor.tensor, approximate=self.approximate)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result)
        return result

    @torch.compile
    @staticmethod
    def gelu_derivative(x: torch.Tensor, grad_output: torch.Tensor, approximate: str) -> torch.Tensor:
        if approximate == "none":
            sqrt_2_pi = 2.5066282749176025  # torch.tensor(2 * torch.pi).sqrt()
            phi_x_cdf = 0.5 * (1 + torch.special.erf(x / 1.4142135381698608))  # torch.sqrt(torch.tensor(2.0))))
            phi_x_pdf = torch.exp(-0.5 * x**2) / sqrt_2_pi
            return (phi_x_cdf + x * phi_x_pdf) * grad_output
        else:
            sqrt_2_over_pi = 0.7978845238685608  # torch.tensor(2.0 / torch.pi).sqrt()
            coeff_cubic = 0.044715
            x2 = x.square()
            inner = x + coeff_cubic * x2 * x
            u = sqrt_2_over_pi * inner
            tanh_u = torch.tanh(u)
            poly = 1 + 3 * coeff_cubic * x2
            return (0.5 * tanh_u + 0.5 * (1 - tanh_u.square()) * (sqrt_2_over_pi * poly * x) + 0.5) * grad_output

class Sigmoid(Module):
    __slots__ = ('graph', '__weakref__')
    def __new__(cls, *, graph=None):
        return super().__new__(cls)

    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def _create_backward(self, input_tensor, result, output_tensor):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = grad_output * output_tensor * (1 - output_tensor)
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.sigmoid(input_tensor.tensor)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result, output_tensor)
        return result

class Tanh(Module):
    __slots__ = ('graph', '__weakref__')
    def __new__(cls, *, graph=None):
        return super().__new__(cls)

    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def _create_backward(self, input_tensor, result, output_tensor):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            grad_input = grad_output * (1 - output_tensor**2)
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.tanh(input_tensor.tensor)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result, output_tensor)
        return result

class Silu(Module):
    __slots__ = ('graph', '__weakref__')
    def __new__(cls, *, graph=None):
        return super().__new__(cls)

    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def _create_backward(self, input_tensor, result, output_tensor):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            grad_output = result_ref.tensor.grad
            s_input_tensor = output_tensor / input_ref.tensor
            grad_input = grad_output * (s_input_tensor + output_tensor * (1 - s_input_tensor))
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    def forward(self, input_tensor):
        output_tensor = F.silu(input_tensor.tensor)
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result, output_tensor)
        return result

class Swish(Module):
    # TODO: implement in future
    __slots__ = ('graph', 'B', 'B_initial', '__weakref__')
    def __new__(cls, *, B_initial=1.0, graph=None):
        assert B_initial > 0
        return super().__new__(cls)

    def __init__(self, *, B_initial=1.0, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None
        self.B = CustomTensor([B_initial], _custom_requires_grad=True, graph=graph, is_leaf=True)
        self.B_initial = B_initial

    def _create_backward(self, input_tensor, result, output_tensor):
        """Creates the _backward hook for result tensor"""
        input_ref = weakref.proxy(input_tensor)
        result_ref = weakref.proxy(result)
        B_ref = weakref.proxy(self.B)

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()
            if B_ref.tensor.grad is None:
                B_ref._zero_grad()
            grad_input, grad_B = self._calculate_gradients(input_ref.tensor, result_ref.tensor, output_tensor, B_ref.tensor)
            # grad_output = result_ref.tensor.grad
            # sig_B_x = output_tensor / input_ref.tensor
            # common = sig_B_x * (1 - sig_B_x) * grad_output

            # grad_input = sig_B_x * grad_output + input_ref.tensor * B_ref.tensor * common
            # grad_B = input_ref.tensor.square() * common
            input_ref.tensor.grad.add_(grad_input)
            B_ref.tensor.grad.add_(grad_B)

        return _backward

    def forward(self, input_tensor):
        scale = self.B.tensor.item()
        output_tensor = F.silu(scale * input_tensor.tensor) / scale
        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(output_tensor, _custom_requires_grad=True, graph=self.graph,due_to_operation=True, is_leaf=False)
        self.graph.add_edge(input_tensor._node_id, result._node_id)
        self.graph.add_edge(self.B._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, result, output_tensor)
        return result

    @torch.compile
    def _calculate_gradients(self, input_tensor, result, output_tensor, B_tensor):
        grad_output =result.grad
        sig_B_x = output_tensor / input_tensor
        common = sig_B_x * (1 - sig_B_x) * grad_output
        grad_input = sig_B_x * grad_output + input_tensor * B_tensor * common
        grad_B = input_tensor.square() * common
        grad_B = grad_B.sum()
        return grad_input, grad_B




In [9]:
# @title Losses
import weakref
import torch
import torch.nn.functional as F

# TODO Lone MSE , MSE with softmax, MSE with sigmoid, cross entropy with softmax, binary cross entropy with sigmoid
class MSE(Module):
    __slots__ = ('graph','__weakref__')
    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def forward(self, input_tensor, target_tensor, weight=None):
        input_t = input_tensor.tensor
        target_t = target_tensor.tensor

        if weight is None:
            loss = F.mse_loss(input_t, target_t, reduction='mean')
        else:
            weight_t = weight
            squared_error = (input_t - target_t) ** 2

            if weight_t.shape == input_t.shape:
                # Per-pixel weight
                weighted_error = weight_t * squared_error
                loss = weighted_error.sum() / weight_t.sum()

            elif weight_t.ndim == 1 and weight_t.shape[0] == input_t.shape[1]:
                # Per-class weight
                dims_to_add = [1] * (input_t.ndim - 2)
                weight_t = weight_t.view(1, -1, *dims_to_add)
                weighted_error = weight_t * squared_error
                loss = weighted_error.sum() / weight_t.sum()

            else:
                raise ValueError(f"Unsupported weight shape: {weight_t.shape}")

        if not self.training:
            return CustomTensor(loss, due_to_operation=True)

        result = CustomTensor(
            loss,
            _custom_requires_grad=True,
            graph=self.graph,
            due_to_operation=True,
            is_leaf=False
        )

        if self.graph is not None:
            self.graph.add_edge(input_tensor._node_id, result._node_id)
            result._backward = self._create_backward(input_tensor, target_tensor, weight)

        return result

    def _create_backward(self, input_tensor, target_tensor, weight):
        input_ref = weakref.proxy(input_tensor)
        target_ref = weakref.proxy(target_tensor)
        weight_ref = weight if weight is not None else None

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()

            grad_input = self._calculate_input_grad(
                input_ref.tensor,
                target_ref.tensor,
                weight_ref
            )
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    @torch.compile
    def _calculate_input_grad(self, input_t, target_t, weight):
        diff = input_t - target_t
        if weight is None:
            return (2 * diff) / input_t.numel()

        if weight.shape == input_t.shape:
            return (2 * weight * diff) / weight.sum()

        elif weight.ndim == 1 and weight.shape[0] == input_t.shape[1]:
            dims_to_add = [1] * (input_t.ndim - 2)
            weight = weight.view(1, -1, *dims_to_add)
            return (2 * weight * diff) / weight.sum()

        else:
            raise ValueError(f"Unsupported weight shape in backward: {weight.shape}")

class CrossEntropyLoss(Module):
    __slots__ = ('graph','__weakref__')
    def __init__(self, *, graph=None):
        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def forward(self, input_tensor, target_tensor, weight= None):

        output_tensor = F.cross_entropy(
            input_tensor.tensor,
            target_tensor.tensor,
            reduction='mean',
            weight=weight
        )

        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)

        result = CustomTensor(
            output_tensor,
            _custom_requires_grad=True,
            graph=self.graph,
            due_to_operation=True,
            is_leaf=False
        )

        self.graph.add_edge(input_tensor._node_id, result._node_id)
        result._backward = self._create_backward(input_tensor, target_tensor, weight)
        return result



    def _create_backward(self, input_tensor, target_tensor,
                        weight):
        input_ref = weakref.proxy(input_tensor)
        target_ref = weakref.proxy(target_tensor)
        weight_ref = weight

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()

            grad_input = self._calculate_input_grad(
                input_ref.tensor,
                target_ref.tensor,
                weight_ref
            )
            input_ref.tensor.grad.add_(grad_input)

        return _backward

    @torch.compile
    def _calculate_input_grad(self, input_tensor, target_tensor,
                             weight):
        batch_size = input_tensor.size(0)
        num_classes = input_tensor.size(1)

        target_one_hot = F.one_hot(target_tensor, num_classes=num_classes).to(input_tensor.dtype)

        softmax_probs = F.softmax(input_tensor, dim=1)

        grad = softmax_probs - target_one_hot

        if weight is not None:
            sample_weights = weight[target_tensor].view(-1, 1)
            grad = grad * sample_weights
            normalizer = sample_weights.sum()
        else:
            normalizer = batch_size
        grad = grad / normalizer
        return grad

class BCEWithLogitsLoss(Module):
    __slots__ = ('graph','__weakref__')
    def __init__(self, *, graph=None):

        super().__init__()
        self.graph = weakref.proxy(graph) if graph is not None else None

    def forward(self, input_tensor, target_tensor, weight= None):
        output_tensor = F.binary_cross_entropy_with_logits(
            input_tensor.tensor,
            target_tensor.tensor,
            reduction='mean',
            pos_weight=weight
        )

        if not self.training:
            return CustomTensor(output_tensor, due_to_operation=True)


        result = CustomTensor(
            output_tensor,
            _custom_requires_grad=True,
            graph=self.graph,
            due_to_operation=True,
            is_leaf=False
        )

        if self.graph is not None:
            self.graph.add_edge(input_tensor._node_id, result._node_id)
            result._backward = self._create_backward(input_tensor, target_tensor, weight)

        return result

    def _create_backward(self, input_tensor, target_tensor, weight):

        input_ref = weakref.proxy(input_tensor)
        target_ref = weakref.proxy(target_tensor)
        weight_ref = weight

        def _backward():
            if input_ref.tensor.grad is None:
                input_ref._zero_grad()

            grad_input = self._calculate_input_grad(
                input_ref.tensor,
                target_ref.tensor,
                weight_ref
            )


            input_ref.tensor.grad.add_(grad_input)

        return _backward

    @torch.compile
    def _calculate_input_grad(self, input_tensor, target_tensor, weight):
        sigmoid_input = torch.sigmoid(input_tensor)

        grad = (sigmoid_input - target_tensor) / input_tensor.numel()

        if weight is not None:
            # pos_weight affects the positive class term (where target == 1)
            # The gradient becomes: (sigmoid - target) * weight / num_elements for positive targets
            # For negative targets, it remains: sigmoid / num_elements
            # This matches PyTorch's implementation of pos_weight in BCEWithLogitsLoss
            weight_factor = torch.where(target_tensor == 1, weight, 1.0)
            grad = grad * weight_factor

        return grad

In [10]:
# @title dataloaders
print("Available splits:", dataset.keys())
train_dataset = iNaturalistDataset(dataset, transform=train_transform, split='train')
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


val_dataset = iNaturalistDataset(dataset, transform=val_transform, split='validation')
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)
# print(f"Training samples: {len(train_loader.dataset)}")

# # Get a sample batch
# sample_batch = next(iter(train_loader))
# images, labels = sample_batch
# print(f"Batch shape: {images.shape}")
# print(f"Labels shape: {labels.shape}")
# print(f"Image dtype: {images.dtype}")
# print(f"Labels dtype: {labels.dtype}")
# try:
#     if hasattr(dataset[list(dataset.keys())[0]], 'features'):
#         features = dataset[list(dataset.keys())[0]].features
#         if 'label' in features:
#             class_names = features['label'].names
#             print(f"Number of classes: {len(class_names)}")
#             print(f"First few classes: {class_names[:5]}")
# except:
#     print("Class information not readily available")

Available splits: dict_keys(['train', 'validation'])


In [11]:
# @title calculation of input output size for the network
# Calculations for CNN Architecture
# Based on the provided code and hyperparameters, here are the detailed calculations for the dimensions of each layer in your CNN. The calculations assume an input image shape of 224x224 with 3 color channels (RGB).

# Layer 1: Convolutional Block 1
# Input: 224x224x3

# Conv2D Layer: (3, 3) filter, num_filters=32. The Keras Conv2D layer with default parameters uses a stride of 1, a padding of 0 (valid), and a dilation rate of 1.

# Output size:

# W_out = (224 - 3 + 2*0)/1 + 1 = 222

# H_out = (224 - 3 + 2*0)/1 + 1 = 222

# Output shape: 222x222x32

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (222 - 2 + 2*0)/2 + 1 = 111

# H_out = (222 - 2 + 2*0)/2 + 1 = 111

# Output shape: 111x111x32

# Layer 2: Convolutional Block 2
# Input: 111x111x32

# Conv2D Layer: (3, 3) filter, num_filters=64. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (111 - 3 + 2*0)/1 + 1 = 109

# H_out = (111 - 3 + 2*0)/1 + 1 = 109

# Output shape: 109x109x64

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (109 - 2 + 2*0)/2 + 1 = 54.5 -> floor(54.5) + 1 = 54

# H_out = (109 - 2 + 2*0)/2 + 1 = 54.5 -> floor(54.5) + 1 = 54

# Output shape: 54x54x64

# Layer 3: Convolutional Block 3
# Input: 54x54x64

# Conv2D Layer: (3, 3) filter, num_filters=128. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (54 - 3 + 2*0)/1 + 1 = 52

# H_out = (54 - 3 + 2*0)/1 + 1 = 52

# Output shape: 52x52x128

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (52 - 2 + 2*0)/2 + 1 = 26

# H_out = (52 - 2 + 2*0)/2 + 1 = 26

# Output shape: 26x26x128

# Layer 4: Convolutional Block 4
# Input: 26x26x128

# Conv2D Layer: (5, 5) filter, num_filters=256. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (26 - 5 + 2*0)/1 + 1 = 22

# H_out = (26 - 5 + 2*0)/1 + 1 = 22

# Output shape: 22x22x256

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (22 - 2 + 2*0)/2 + 1 = 11

# H_out = (22 - 2 + 2*0)/2 + 1 = 11

# Output shape: 11x11x256

# Layer 5: Convolutional Block 5
# Input: 11x11x256

# Conv2D Layer: (7, 7) filter, num_filters=512. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (11 - 7 + 2*0)/1 + 1 = 5

# H_out = (11 - 7 + 2*0)/1 + 1 = 5

# Output shape: 5x5x512

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (5 - 2 + 2*0)/2 + 1 = 2.5 -> floor(2.5) + 1 = 2

# H_out = (5 - 2 + 2*0)/2 + 1 = 2.5 -> floor(2.5) + 1 = 2

# Output shape: 2x2x512

# Final Dense Layer
# Flatten Layer: The output of the last max-pooling layer is a 2x2x512 tensor. The flatten layer reshapes this into a one-dimensional vector.

# Vector size: 2 * 2 * 512 = 2048

# Dense Layer: The input to the first dense layer will be a vector of size 2048.

In [20]:
# @title pytorch CNN model equivalent
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBNActMaxPool(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size_conv, stride_conv, padding_conv,
                 kernel_size_pool, stride_pool, padding_pool):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size_conv,
            stride=stride_conv,
            padding=padding_conv,
        )
        self.batchnorm = nn.BatchNorm2d(num_features=out_channels, eps=1e-5, momentum=0.1)
        self.activation = nn.GELU(approximate='tanh')  # PyTorch ≥ 1.13
        self.maxpool = nn.MaxPool2d(
            kernel_size=kernel_size_pool,
            stride=stride_pool,
            padding=padding_pool,
            dilation=1
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.activation(x)
        x = self.maxpool(x)
        return x


class CNN_Model_py(nn.Module):
    def __init__(self):
        super().__init__()
        # Input shape: (3, 224, 224)
        self.layer1 = ConvBNActMaxPool(3, 32, 3, 1, 0, 2, 2, 0)     # -> (32, 111, 111)
        self.layer2 = ConvBNActMaxPool(32, 64, 3, 1, 0, 2, 2, 0)    # -> (64, 54, 54)
        self.layer3 = ConvBNActMaxPool(64, 128, 3, 1, 0, 2, 2, 0)   # -> (128, 26, 26)
        self.layer4 = ConvBNActMaxPool(128, 256, 5, 1, 0, 2, 2, 0)  # -> (256, 11, 11)
        self.layer5 = ConvBNActMaxPool(256, 512, 7, 1, 0, 2, 2, 0)  # -> (512, 2, 2)

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(512 * 2 * 2, 512)
        self.relu = nn.ReLU()
        self.output = nn.Linear(512, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.flatten(x)       # -> (B, 2048)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.output(x)
        return x


In [34]:
# @title Defining the custom CNN Network
# Conv -> Batch Normalization -> Activation -> MaxPool.
class conv_batchnorm_activation_maxpool(Module):
  def __init__(self, in_channels, out_channels, kernel_size_conv, stride_conv, padding_conv,kernel_size_pool,stride_pool,padding_pool, graph):
    super().__init__()
    self.conv = Conv2d(
        in_channels = in_channels,
        out_channels = out_channels,
        kernel_size = kernel_size_conv,
        stride = stride_conv,
        padding = padding_conv,
        graph = graph,
        activation="gelu_approx"
    )
    self.batchnorm = BatchNorm_Nd(
        num_features=out_channels,
        eps=1e-5,
        momentum=0.1,
        graph=graph)
    self.activation = GeLu(
        approximate='tanh',
        graph=graph)
    self.maxpool = MaxPool2d(
        kernel_size=kernel_size_pool,#(2,2),
        stride=stride_pool,#(2,2)
        padding=padding_pool,#(0,0)
        dilation=1,
        graph=graph)
  def forward(self, x):
    x = self.conv(x)
    x = self.batchnorm(x)
    x = self.activation(x)
    x = self.maxpool(x)
    return x
class CNN_Model(Module):
  # configaration kernel size  [(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]
  # output_channels [32,64,128,256,512]
  # dense layer 512

  # conv 224-3 +1 = 222
  # max 222 -2/2 +1 = 111

  # conv 111-3 +1 = 109
  # max 109 -2/2 +1 = 54

  # conv 54 -3 +1 = 52
  # max 52 -2/2 +1 = 26

  # conv 26 -5 +1 = 22
  # max 22-2/2 +1 =11

  # conv 11 -7 +1 = 5
  # max = 5 -2/2 +1 = 2

  # hence for linear layer the reshaped tensor is 512*2*2 = 2048

  def __init__(self,graph):
    super().__init__()
    #in_channels, out_channels, kernel_size_conv, stride_conv, padding_conv,kernel_size_pool,stride_pool,padding_pool, graph
    self.layer1 = conv_batchnorm_activation_maxpool(3,32, 3,1,0, 2,2,0, graph)
    self.layer2 = conv_batchnorm_activation_maxpool(32,64, 3,1,0, 2,2,0, graph)
    self.layer3 = conv_batchnorm_activation_maxpool(64,128, 3,1,0, 2,2,0, graph)
    self.layer4 = conv_batchnorm_activation_maxpool(128,256, 5,1,0, 2,2,0, graph)
    self.layer5 = conv_batchnorm_activation_maxpool(256,512, 7,1,0, 2,2,0, graph)
    self.linear1 = Linear(
        in_features=512*2*2,
        out_features=512,
        graph=graph,
        activation="relu"
    )
    self.ac1 = ReLu(graph=graph)
    self.output = Linear(
        in_features=512,
        out_features=10,
        graph=graph,
        activation="relu"
    )
  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = x.reshape((x.shape[0],-1))
    x = self.linear1(x)
    x = self.ac1(x)
    x = self.output(x)
    return x





In [21]:
cnn_model_pytorch = CNN_Model_py()

In [23]:
cnn_model_pytorch.state_dict().keys()

odict_keys(['layer1.conv.weight', 'layer1.conv.bias', 'layer1.batchnorm.weight', 'layer1.batchnorm.bias', 'layer1.batchnorm.running_mean', 'layer1.batchnorm.running_var', 'layer1.batchnorm.num_batches_tracked', 'layer2.conv.weight', 'layer2.conv.bias', 'layer2.batchnorm.weight', 'layer2.batchnorm.bias', 'layer2.batchnorm.running_mean', 'layer2.batchnorm.running_var', 'layer2.batchnorm.num_batches_tracked', 'layer3.conv.weight', 'layer3.conv.bias', 'layer3.batchnorm.weight', 'layer3.batchnorm.bias', 'layer3.batchnorm.running_mean', 'layer3.batchnorm.running_var', 'layer3.batchnorm.num_batches_tracked', 'layer4.conv.weight', 'layer4.conv.bias', 'layer4.batchnorm.weight', 'layer4.batchnorm.bias', 'layer4.batchnorm.running_mean', 'layer4.batchnorm.running_var', 'layer4.batchnorm.num_batches_tracked', 'layer5.conv.weight', 'layer5.conv.bias', 'layer5.batchnorm.weight', 'layer5.batchnorm.bias', 'layer5.batchnorm.running_mean', 'layer5.batchnorm.running_var', 'layer5.batchnorm.num_batches_tra

In [24]:

with AutogradGraph() as graph:
  cnn_model = CNN_Model(graph=graph)


In [32]:
cnn_model._modules

OrderedDict([('layer1',
              <__main__.conv_batchnorm_activation_maxpool at 0x7a821232e490>),
             ('layer2',
              <__main__.conv_batchnorm_activation_maxpool at 0x7a82106d9db0>),
             ('layer3',
              <__main__.conv_batchnorm_activation_maxpool at 0x7a82106d9360>),
             ('layer4',
              <__main__.conv_batchnorm_activation_maxpool at 0x7a821232df90>),
             ('layer5',
              <__main__.conv_batchnorm_activation_maxpool at 0x7a821232d5e0>),
             ('linear1', <__main__.Linear at 0x7a82128caff0>),
             ('ac1', <__main__.ReLu at 0x7a821225e4d0>),
             ('output', <__main__.Linear at 0x7a82128cb060>)])

In [39]:
# @title Verifying that forward pass works the same or not

with AutogradGraph() as graph, torch.inference_mode():
  cnn_model_pytorch = CNN_Model_py()
  cnn_model = CNN_Model(graph=graph)
  # copying all the weights of pytorch to our model
  # assume cnn_model_pytorch (a trained torch CNN_Model_py) and
  # cnn_model (your custom CNN_Model under AutogradGraph) are already instantiated
  cnn_model_pytorch.layer1.conv.weight.data.copy_(
    cnn_model._modules['layer1'].conv.weight.tensor
  )
  cnn_model_pytorch.layer1.conv.bias.data.copy_(
      cnn_model._modules['layer1'].conv.bias.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.weight.data.copy_(
      cnn_model._modules['layer1'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.bias.data.copy_(
      cnn_model._modules['layer1'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer1'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer1.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer1'].batchnorm.running_var
  )

  #### Layer 2 ####
  cnn_model_pytorch.layer2.conv.weight.data.copy_(
      cnn_model._modules['layer2'].conv.weight.tensor
  )
  cnn_model_pytorch.layer2.conv.bias.data.copy_(
      cnn_model._modules['layer2'].conv.bias.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.weight.data.copy_(
      cnn_model._modules['layer2'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.bias.data.copy_(
      cnn_model._modules['layer2'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer2'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer2.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer2'].batchnorm.running_var
  )

  #### Layer 3 ####
  cnn_model_pytorch.layer3.conv.weight.data.copy_(
      cnn_model._modules['layer3'].conv.weight.tensor
  )
  cnn_model_pytorch.layer3.conv.bias.data.copy_(
      cnn_model._modules['layer3'].conv.bias.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.weight.data.copy_(
      cnn_model._modules['layer3'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.bias.data.copy_(
      cnn_model._modules['layer3'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer3'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer3.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer3'].batchnorm.running_var
  )

  #### Layer 4 ####
  cnn_model_pytorch.layer4.conv.weight.data.copy_(
      cnn_model._modules['layer4'].conv.weight.tensor
  )
  cnn_model_pytorch.layer4.conv.bias.data.copy_(
      cnn_model._modules['layer4'].conv.bias.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.weight.data.copy_(
      cnn_model._modules['layer4'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.bias.data.copy_(
      cnn_model._modules['layer4'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer4'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer4.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer4'].batchnorm.running_var
  )

  #### Layer 5 ####
  cnn_model_pytorch.layer5.conv.weight.data.copy_(
      cnn_model._modules['layer5'].conv.weight.tensor
  )
  cnn_model_pytorch.layer5.conv.bias.data.copy_(
      cnn_model._modules['layer5'].conv.bias.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.weight.data.copy_(
      cnn_model._modules['layer5'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.bias.data.copy_(
      cnn_model._modules['layer5'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer5'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer5.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer5'].batchnorm.running_var
  )
  #### Linear 1 ####
  cnn_model_pytorch.linear1.weight.data.copy_(
      cnn_model._modules['linear1'].weight.tensor
  )
  cnn_model_pytorch.linear1.bias.data.copy_(
      cnn_model._modules['linear1'].bias.tensor
  )
  #### Output ####
  cnn_model_pytorch.output.weight.data.copy_(
      cnn_model._modules['output'].weight.tensor
  )
  cnn_model_pytorch.output.bias.data.copy_(
      cnn_model._modules['output'].bias.tensor
  )



  cnn_model.eval()
  cnn_model_pytorch.eval()
  sample_input_torch = torch.randn(3,3,224,224)
  sample_input = CustomTensor(sample_input_torch.clone(),_custom_requires_grad=False,graph=None)
  output = cnn_model(sample_input)
  output_torch = cnn_model_pytorch(sample_input_torch)
  torch.allclose(output.tensor,output_torch)



In [41]:
del cnn_model,cnn_model_pytorch,sample_input,sample_input_torch

In [52]:
# @title Verifying that backward pass works the same or not
# .data.copy_ only copies the actual data in memory of the tensor not the autograd metadata associated with the tensor
import time
with AutogradGraph() as graph:
  cnn_model_pytorch = CNN_Model_py()
  cnn_model = CNN_Model(graph=graph)
  # copying all the weights of pytorch to our model
  # assume cnn_model_pytorch (a trained torch CNN_Model_py) and
  # cnn_model (your custom CNN_Model under AutogradGraph) are already instantiated
  cnn_model_pytorch.layer1.conv.weight.data.copy_(
    cnn_model._modules['layer1'].conv.weight.tensor
  )
  cnn_model_pytorch.layer1.conv.bias.data.copy_(
      cnn_model._modules['layer1'].conv.bias.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.weight.data.copy_(
      cnn_model._modules['layer1'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.bias.data.copy_(
      cnn_model._modules['layer1'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer1.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer1'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer1.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer1'].batchnorm.running_var
  )

  #### Layer 2 ####
  cnn_model_pytorch.layer2.conv.weight.data.copy_(
      cnn_model._modules['layer2'].conv.weight.tensor
  )
  cnn_model_pytorch.layer2.conv.bias.data.copy_(
      cnn_model._modules['layer2'].conv.bias.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.weight.data.copy_(
      cnn_model._modules['layer2'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.bias.data.copy_(
      cnn_model._modules['layer2'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer2.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer2'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer2.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer2'].batchnorm.running_var
  )

  #### Layer 3 ####
  cnn_model_pytorch.layer3.conv.weight.data.copy_(
      cnn_model._modules['layer3'].conv.weight.tensor
  )
  cnn_model_pytorch.layer3.conv.bias.data.copy_(
      cnn_model._modules['layer3'].conv.bias.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.weight.data.copy_(
      cnn_model._modules['layer3'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.bias.data.copy_(
      cnn_model._modules['layer3'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer3.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer3'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer3.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer3'].batchnorm.running_var
  )

  #### Layer 4 ####
  cnn_model_pytorch.layer4.conv.weight.data.copy_(
      cnn_model._modules['layer4'].conv.weight.tensor
  )
  cnn_model_pytorch.layer4.conv.bias.data.copy_(
      cnn_model._modules['layer4'].conv.bias.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.weight.data.copy_(
      cnn_model._modules['layer4'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.bias.data.copy_(
      cnn_model._modules['layer4'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer4.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer4'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer4.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer4'].batchnorm.running_var
  )

  #### Layer 5 ####
  cnn_model_pytorch.layer5.conv.weight.data.copy_(
      cnn_model._modules['layer5'].conv.weight.tensor
  )
  cnn_model_pytorch.layer5.conv.bias.data.copy_(
      cnn_model._modules['layer5'].conv.bias.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.weight.data.copy_(
      cnn_model._modules['layer5'].batchnorm.weight.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.bias.data.copy_(
      cnn_model._modules['layer5'].batchnorm.bias.tensor
  )
  cnn_model_pytorch.layer5.batchnorm.running_mean.data.copy_(
      cnn_model._modules['layer5'].batchnorm.running_mean
  )
  cnn_model_pytorch.layer5.batchnorm.running_var.data.copy_(
      cnn_model._modules['layer5'].batchnorm.running_var
  )
  #### Linear 1 ####
  cnn_model_pytorch.linear1.weight.data.copy_(
      cnn_model._modules['linear1'].weight.tensor
  )
  cnn_model_pytorch.linear1.bias.data.copy_(
      cnn_model._modules['linear1'].bias.tensor
  )
  #### Output ####
  cnn_model_pytorch.output.weight.data.copy_(
      cnn_model._modules['output'].weight.tensor
  )
  cnn_model_pytorch.output.bias.data.copy_(
      cnn_model._modules['output'].bias.tensor
  )



  cnn_model.train()
  cnn_model_pytorch.train()

  sample_input_torch = torch.randn(3,3,224,224)
  sample_input = CustomTensor(sample_input_torch.clone(),_custom_requires_grad=True,graph=graph)
  output = cnn_model(sample_input)
  output_torch = cnn_model_pytorch(sample_input_torch)
  loss = output.sum()
  loss_torch = output_torch.sum()
  st = time.time()
  loss.backward()
  et = time.time()
  print(f"Shitty implementation backward Takes {et-st} seconds")
  st = time.time()
  loss_torch.backward()
  et = time.time()
  print(f"Pytorch implementation backward Takes {et-st} seconds")


Shitty implementation backward Takes 0.5528960227966309 seconds
Pytorch implementation backward Takes 0.3487536907196045 seconds


In [51]:
import torch

def compare_grads(param1, param2, name):
    grad1 = param1.grad
    grad2 = param2.grad
    if grad1 is None or grad2 is None:
        print(f"{name}: One of the gradients is None.")
    else:
        equal = torch.allclose(grad1, grad2, atol=1e-4)
        print(f"{name}: {'✅ Same' if equal else '❌ Different'}")

# Layer-wise comparisons
for i in range(1, 6):
    layer = f"layer{i}"
    torch_layer = getattr(cnn_model_pytorch, layer)
    custom_layer = cnn_model._modules[layer]

    compare_grads(torch_layer.conv.weight, custom_layer.conv.weight.tensor, f"{layer}.conv.weight")
    compare_grads(torch_layer.conv.bias, custom_layer.conv.bias.tensor, f"{layer}.conv.bias")
    compare_grads(torch_layer.batchnorm.weight, custom_layer.batchnorm.weight.tensor, f"{layer}.batchnorm.weight")
    compare_grads(torch_layer.batchnorm.bias, custom_layer.batchnorm.bias.tensor, f"{layer}.batchnorm.bias")

# Linear 1
compare_grads(cnn_model_pytorch.linear1.weight, cnn_model._modules['linear1'].weight.tensor, "linear1.weight")
compare_grads(cnn_model_pytorch.linear1.bias, cnn_model._modules['linear1'].bias.tensor, "linear1.bias")

# Output
compare_grads(cnn_model_pytorch.output.weight, cnn_model._modules['output'].weight.tensor, "output.weight")
compare_grads(cnn_model_pytorch.output.bias, cnn_model._modules['output'].bias.tensor, "output.bias")


layer1.conv.weight: ❌ Different
layer1.conv.bias: ✅ Same
layer1.batchnorm.weight: ❌ Different
layer1.batchnorm.bias: ❌ Different
layer2.conv.weight: ❌ Different
layer2.conv.bias: ✅ Same
layer2.batchnorm.weight: ❌ Different
layer2.batchnorm.bias: ❌ Different
layer3.conv.weight: ❌ Different
layer3.conv.bias: ✅ Same
layer3.batchnorm.weight: ✅ Same
layer3.batchnorm.bias: ✅ Same
layer4.conv.weight: ✅ Same
layer4.conv.bias: ✅ Same
layer4.batchnorm.weight: ✅ Same
layer4.batchnorm.bias: ✅ Same
layer5.conv.weight: ✅ Same
layer5.conv.bias: ✅ Same
layer5.batchnorm.weight: ✅ Same
layer5.batchnorm.bias: ✅ Same
linear1.weight: ✅ Same
linear1.bias: ✅ Same
output.weight: ✅ Same
output.bias: ✅ Same


In [40]:
torch.allclose(output.tensor,output_torch)

True

In [ ]:
# class SimpleNet(Module):
#     def __init__(self, graph):
#         super().__init__()
#         self.layer1 = Linear(3, 4, graph=graph)
#         self.activation = ReLu(graph=graph)
#         self.layer2 = Linear(4, 2, graph=graph)

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.activation(x)
#         x = self.layer2(x)
#         return x

In [4]:
class_names = {
    '0': 'Amphibia',
    '1': 'Animalia',
    '2': 'Arachnida',
    '3': 'Aves',
    '4': 'Fungi',
    '5': 'Insecta',
    '6': 'Mammalia',
    '7': 'Mollusca',
    '8': 'Plantae',
    '9': 'Reptilia'
}

print(class_names)

{'0': 'Amphibia', '1': 'Animalia', '2': 'Arachnida', '3': 'Aves', '4': 'Fungi', '5': 'Insecta', '6': 'Mammalia', '7': 'Mollusca', '8': 'Plantae', '9': 'Reptilia'}
